# OpenAI Responses API
https://platform.openai.com/docs/api-reference/responses  
배포된 openai의 api key를 .env의 OPENAI_API_KEY에 등록하여 사용합니다.

In [1]:
import requests
from pprint import pprint
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
URL = "https://api.openai.com/v1/responses"
model = "gpt-4o-mini"

### Chat Completions vs Responses 차이점

1. 상태 유지 방식  
   Chat Completions는 매번 전체 대화 내역을 리스트로 묶어 보내야 하지만, Responses API는 previous_response_id를 통해 서버 측에서 이전 응답을 참조하는 상태 중심(Stateful) 상호작용을 지원한다.
2. 입력 파라미터  
   기존의 messages 필드가 input 필드로 대체된다.  
   input은 단순 문자열이나 이미지, 파일 등 다양한 형태를 포함하는 배열로 구성될 수 있다.
3. 시스템 메시지  
   role: system 방식 대신 instructions라는 별도의 최상위 파라미터를 사용하여 모델의 페르소나와 지침을 설정한다.
4. 응답 구조  
   결과 데이터가 choices 대신 output 배열에 담겨 반환된다. 
5. 내장 도구  
   별도의 세션 관리 없이도 웹 검색(Web Search), 파일 검색(File Search) 등의 도구를 더욱 직접적으로 활용할 수 있다.

### REST API 요청
라이브러리 없이 직접 HTTP 통신을 통해 /v1/responses 엔드포인트를 호출한다.

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {OPENAI_API_KEY}"
}

payload = {
    "model": model,
    "instructions": "당신은 친절한 AI 강사입니다.",
    "input": "Responses API가 뭐야? 2~3문장으로 답변해줘",
    # 이전 방식
    # 'message' : [
    #     {'role' : 'system', 'content' : '당신은 강사입니다.'},
    #     {'role' : 'user', 'content' : 'api가 뭐야?'}
    # ]
}

response = requests.post(URL, headers=headers, json=payload)
res_data = response.json()
pprint(res_data)

if "output" in res_data:
    # output 배열 내부의 첫 번째 메시지 텍스트 추출
    content_text = res_data["output"][0]["content"][0]["text"]
    print(f"\n답변: {content_text}")

{'background': False,
 'billing': {'payer': 'developer'},
 'completed_at': 1768975831,
 'created_at': 1768975828,
 'error': None,
 'frequency_penalty': 0.0,
 'id': 'resp_03d23c93cb19c8f80069706dd4dea8819097544844e68e9d03',
 'incomplete_details': None,
 'instructions': '당신은 친절한 AI 강사입니다.',
 'max_output_tokens': None,
 'max_tool_calls': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'logprobs': [],
                          'text': 'Responses API는 개발자들이 애플리케이션에서 사용자 요청에 대한 '
                                  '응답을 자동으로 처리할 수 있도록 도와주는 도구입니다. 이를 통해 API '
                                  '호출에 대한 결과를 손쉽게 응답 형식으로 정의하고, 사용자에게 필요한 정보를 '
                                  '제공할 수 있습니다. 이 API는 데이터 통신의 편리성을 높여 줍니다.',
                          'type': 'output_text'}],
             'id': 'msg_03d23c93cb19c8f80069706dd563a48190a5d7b9936d050a65',
             'role': 'assistant',
             'statu

### OpenAI SDK를 활용한 요청
client.responses 리소스를 사용하여 효율적으로 응답을 생성합니다.

In [3]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.responses.create(
    model=model,
    input="OpenAI Responses API를 사용하면 어떤 점이 좋아?"
)

# SDK 전용 속성인 output_text를 사용하여 답변 확인
print(response.output_text)

OpenAI의 Responses API를 사용하면 다음과 같은 장점이 있습니다:

1. **자연어 처리**: 인간과 유사한 수준의 자연어 이해 및 생성을 통해 다양한 질문과 대화에 응답할 수 있습니다.

2. **다양한 용도**: 고객 지원, 콘텐츠 생성, 교육 자료, 코드 작성 등 여러 분야에서 활용할 수 있습니다.

3. **사용자 맞춤형**: 특정 요구사항이나 스타일에 맞추어 응답을 조정할 수 있어, 개인화된 경험을 제공합니다.

4. **신속한 처리**: 대량의 데이터를 빠르게 처리하고 응답할 수 있어, 실시간 인터랙티브 애플리케이션에 적합합니다.

5. **확장성**: 비즈니스의 요구에 맞춰 쉽게 확장할 수 있고, 사용량에 따라 유연하게 조정할 수 있습니다.

6. **응답 품질**: 최신 모델을 사용하여 높은 품질의 응답을 생성하며, 지속적으로 개선되고 있습니다.

이런 장점들 덕분에 OpenAI Responses API는 다양한 애플리케이션에 eficaz하게 통합될 수 있습니다.


### Instructions (System Prompt) 비교
기존의 system role 메시지 대신 instructions 파라미터에 지침을 입력한다.

In [4]:
user_input = "아침 일찍 일어나는 습관의 장점에 대해 말해줘."

personas = {
    "열정적인 셰프": "당신은 요리에 인생을 건 셰프입니다. 인생의 모든 이치를 요리 과정과 재료에 비유하여 설명하세요.",
    "엄격한 헬스 트레이너": "당신은 매우 엄격한 운동 전문가입니다. 강한 어조로 자기관리를 강조하며 답변하세요.",
    "지혜로운 판다": "당신은 대나무 숲에 사는 느긋하고 지혜로운 판다입니다. 느릿느릿하고 평화로운 말투로 조언을 건네세요."
}

for name, inst in personas.items():
    print(f"--- [{name}] 버전 ---")
    response = client.responses.create(
        model=model,
        instructions=inst,
        input=user_input
    )
    print(response.output_text)
    print("\n")

--- [열정적인 셰프] 버전 ---
아침 일찍 일어나는 습관을 요리에 비유해 볼게요.

마치 요리를 시작하기 전에 모든 재료를 미리 준비하는 것과 같습니다. 아침 일찍 일어나면 하루를 차분하게 준비할 수 있는 시간을 가질 수 있습니다. 재료를 정리하고, 필요한 것들을 체크하는 동안, 정신적으로도 하루를 세팅하는 효과가 있죠. 이처럼 준비된 마음가짐은 하루를 알차고 효율적으로 만들 수 있습니다.

또한, 아침은 신선한 재료가 제격이듯, 일찍 일어나는 것은 하루의 신선함과 에너지를 최대한 활용할 수 있게 해줍니다. 이른 시간에는 조용하고 방해받지 않는 환경에서 집중할 수 있어, 마치 영감을 주는 양념을 더하는 것처럼 창의적인 아이디어가 떠오르기 좋습니다.

아침 햇살과 함께하는 시간은 마치 요리의 마지막 순간에 신선한 허브를 넣어 향을 더하는 것과 같습니다. 하루의 시작에 긍정적인 에너지를 공급받고, 이를 통해 더욱 맛있는 하루를 만들어 갈 수 있습니다.

마지막으로, 아침 일찍 일어나는 습관은 장기적으로 보면 자신만의 레시피를 만드는 것과 같아요. 매일 조금씩 일찍 일어나는 연습을 통해, 결국 자신에게 맞는 최상의 하루를 조리하는 전문가가 될 수 있답니다.


--- [엄격한 헬스 트레이너] 버전 ---
아침 일찍 일어나는 습관은 성공적인 삶을 위한 필수 요소입니다. 첫째, **시간 관리**가 향상됩니다. 일찍 일어남으로써 하루를 계획하고 목표를 설정할 수 있는 시간을 확보하게 됩니다. 두 번째, **생산성이 증가**합니다. 조용한 아침 시간에는 방해 요소가 적어 집중력을 높이고 중요한 일을 미리 해치울 수 있습니다. 

셋째, **정신적 및 신체적 건강**에 긍정적인 영향을 미칩니다. 아침에는 해가 뜨기 전에 명상이나 운동을 통해 마음을 차분히 하고 몸을 활성화할 수 있습니다. 또한, 규칙적인 수면 패턴은 전반적인 건강을 증진시키고 스트레스를 줄여줍니다.

마지막으로, 아침 일찍 일어나는 습관은 **주도적인 삶**을 살아갈 수 있는 기회를 제공합니다. 대충 

### input 
input에는 text뿐만 아니라 메시지 객체, 이미지, 파일 등이 들어갈 수 있다.  
메시지 객체는 다음과 같은 형식을 가진다.
```
{
  'content' : 컨텐트에 대한 text 또는 list,
  'role' : 'user' / 'assistant' / 'system' / 'developer' 중 하나가 들어간다.
  'type' : 'message'
}
```
https://platform.openai.com/docs/api-reference/responses/create#responses_create-input

In [5]:
response = client.responses.create(
    model=model,
    input=[
    {
        "type": "message",
        "role": "user",
        "content": [
            {"type": "input_text", "text": "이 사진에 대해 설명해줘."},
            {"type": "input_image", "image_url": "https://images.dog.ceo/breeds/shiba/shiba-8.jpg"}
        ]
    }
]
)

print(response.output_text)

이 사진에는 귀여운 개가 서 있는 모습이 보입니다. 이 개는 검정색과 갈색 털을 가진 시바 이누(Shiba Inu)로 보이며, 잘 정돈된 모습을 하고 있습니다. 개는 발을 약간 들어 유쾌한 표정을 짓고 있으며, 주변에는 풀과 자연이 어우러져 있습니다. 그 모습에서 활발하고 친근한 느낌이 전달됩니다.


### Temperature 비교
무작위성을 조절하는 temperature 파라미터는 동일하게 사용 가능하다.

In [ ]:
creative_topic = "운동화 브랜드의 새로운 슬로건을 5개 제안해줘. 단, 기발하게 작성해줘."
temperatures = [0.3, 0.8, 1.0, 1.3, 1.5, 1.6, 1.8]

for t in temperatures:
    print(f"### 설정값 (Temperature): {t} ###")
    response = client.responses.create(
        model=model,
        input=creative_topic,
        temperature=t,
        max_output_tokens=200
    )
    print(response.output_text)
    print("=" * 50)

### previous_response_id를 활용한 대화 맥락 유지
과거 응답의 ID를 전달하여 이전 대화 내용을 자동으로 참조한다.

In [ ]:

print("Q1: 내 이름은 jun이야.")
res1 = client.responses.create(
    model=model,
    instructions="당신은 사용자의 이름을 기억하는 비서입니다.",
    input="내 이름은 jun이야."
)
print(f"A1: {res1.output_text}\n")

response_id = res1.id
print(res1.id)

# 두 번째 대화 (이전 ID 전달)
print("Q2: 난 떡볶이 먹었어")
res2 = client.responses.create(
    model=model,
    input="난 떡볶이 먹었어?",
    previous_response_id=response_id
)
print(f"A2: {res2.output_text}")




Q1: 내 이름은 jun이야.
A1: 안녕하세요, Jun! 어떻게 도와드릴까요?

resp_0ae07f912dc3615500697070feae48819daaf61211a0373f54
Q2: 난 떡볶이 먹었어
A2: 떡볶이 맛있죠! 매운 거 좋아하시나요? 어떤 스타일의 떡볶이를 드셨나요?
Q3
A3: 당신은 Jun이라고 말씀하셨어요! 더 궁금한 점이나 나누고 싶은 이야기가 있다면 말씀해 주세요.


### Structured Outputs (JSON 형식 출력)
text 파라미터의 format 설정을 통해 JSON 출력을 강제할 수 있습니다.

In [11]:
import json

response = client.responses.create(
    model=model,
    instructions="너는 요리사야. 답변은 반드시 JSON 형식으로 해줘.",
    input="떡볶이 레시피 알려줘. json형식으로 대답해줘.",
    text={
        "format": {"type": "json_object"}
    }
)

res_json = json.loads(response.output_text)
pprint(res_json)

{'cooking_time': '20분',
 'ingredients': [{'amount': '300g', 'name': '떡'},
                 {'amount': '100g', 'name': '어묵'},
                 {'amount': '1개', 'name': '양파'},
                 {'amount': '1대', 'name': '대파'},
                 {'amount': '2큰술', 'name': '고추장'},
                 {'amount': '1큰술', 'name': '고춧가루'},
                 {'amount': '1큰술', 'name': '간장'},
                 {'amount': '1큰술', 'name': '설탕'},
                 {'amount': '1작은술', 'name': '참기름'},
                 {'amount': '500ml', 'name': '물'}],
 'instructions': ['물 500ml를 냄비에 붓고 끓입니다.',
                  '떡과 어묵을 넣고 끓입니다.',
                  '떡이 부드러워지면 양파와 대파를 추가합니다.',
                  '고추장, 고춧가루, 간장, 설탕을 넣고 잘 섞습니다.',
                  '소스가 걸쭉해질 때까지 중불에서 끓입니다.',
                  '마지막으로 참기름을 넣고 섞은 후, 불을 끕니다.',
                  '접시에 담고, 좋아하는 고명을 추가하여 뿌리면 완성입니다.'],
 'recipe_name': '떡볶이',
 'serving_size': '2-3인분'}


### Streaming (실시간 응답 처리)
stream=True 설정을 통해 생성되는 즉시 데이터를 수신한다.

In [ ]:
prompt = "양자 역학에 대해 초등학생도 이해할 수 있게 설명해줘."
print(f"질문: {prompt}\n")
print("답변: ", end="")

stream = client.responses.create(
    model=model,
    input=prompt,
    stream=True
)

for chunk in stream:
    # 스트리밍 시 delta 이벤트를 통해 텍스트 조각을 수신함
    # print(chunk)
    if hasattr(chunk, 'delta') and chunk.delta:
        print(chunk.delta, end="", flush=True)

    # try:
        # print(chunk.delta, end="", flush=True)
    # except:
    #     pass

print("\n\n--- 스트리밍 종료 ---")

질문: 양자 역학에 대해 초등학생도 이해할 수 있게 설명해줘.

답변: 양자 역학은 아주 작은 것들, 예를 들어 원자나 전자 같은 것들이 어떻게 움직이고 상호작용하는지를 설명하는 과학이에요. 보통 우리가 아는 물체는 큰 것들이고, 그들은 우리가 쉽게 볼 수 있어요. 하지만 양자 역학은 너무 작아서 눈으로 볼 수 없는 것들에 대해 이야기해요.

이해하기 쉽게 몇 가지 예를 들어볼게요!

1. **공의 움직임**: 우리가 공을 던지면, 그 공은 예측할 수 있게 움직이죠. 하지만 양자 세계에서는 작은 입자들이 마치 **무한한 방법**으로 움직이는 것처럼 느껴져요. 그래서 어떤 때는 어디에 있는지 확실히 알 수 없어요.

2. **상태가 겹치기**: 양자 물체는 동시에 여러 상태에 있을 수 있어요. 마치 동전을 던졌을 때, 공중에 있을 때는 '앞면'과 '뒷면'이 동시에 있는 것처럼요. 하지만 떨어지는 순간에는 '앞면'이나 '뒷면' 중 하나만 나오죠.

3. **관측의 중요성**: 양자 세계에서는 관찰하는 것 자체가 중요해요. 당신이 어떤 것을 보고 있으면 그 상태가 결정되지만, 그렇지 않으면 여러 상태가 동시에 존재할 수 있어요.

이렇게 양자 역학은 우리가 아는 세계와는 조금 다른, 특별한 규칙들이 있는 세계에 대한 이야기예요! 이 작은 세계가 어떻게 작동하는지를 연구하면서 과학자들은 새로운 기술을 발견하고 있어요. 예를 들어, 스마트폰이나 컴퓨터 같은 것들도 양자 역학 덕분에 발전했답니다!

--- 스트리밍 종료 ---


### 비동기 요청
AsyncOpenAI 클라이언트를 사용하여 병렬로 여러 응답을 처리한다.

In [ ]:
from openai import AsyncOpenAI
import asyncio

async_client = AsyncOpenAI(api_key=OPENAI_API_KEY)

async def get_food(city):
    print(f"[{city}] 검색...")
    res = await async_client.responses.create(
        model=model,
        input=f"{city}에서 꼭 먹어야 할 음식 한 가지만 추천해줘."
    )
    return f"{city}: {res.output_text}"

async def main():
    cities = ["서울", "파리", "뉴욕", "도쿄", "방콕", "로마"]
    tasks = [get_food(c) for c in cities]
    results = await asyncio.gather(*tasks)
    for r in results:
        print(r)

await main()

### 추가 관리 기능 (조회, 삭제, 압축)
생성된 응답을 ID로 조회하거나 삭제하고, 긴 대화의 컨텍스트를 압축하는 기능을 제공합니다.

In [25]:

response = client.responses.create(
    model=model,
    input="안녕 반가워!"
)

response_id = response.id
print(response.output_text)

안녕하세요! 반가워요! 어떻게 도와드릴까요?


In [23]:
# 응답 조회

retrieved = client.responses.retrieve(response_id)

print(retrieved.output_text)


안녕! 반가워! 어떻게 도와줄까?


In [24]:
# 응답 삭제
client.responses.delete(response_id)

try:
    # 응답이 삭제가 되서 retrieve할 수 없다.
    client.responses.retrieve(response_id)
except Exception as e:
    print(e)

Error code: 404 - {'error': {'message': "Response with id 'resp_014a0a2b957b7d4d00697072581b688195ad8b5eaa76e39c81' not found.", 'type': 'invalid_request_error', 'param': None, 'code': None}}


In [ ]:
# 대화 압축 (대화가 너무 길어질 때 사용)

result = client.responses.compact(
    model=model, 
    previous_response_id=response_id
    )


CompactedResponse(id='resp_068a92f8b636a821016970761f450081938e41a940a65bedfb', created_at=1768977956, object='response.compaction', output=[ResponseOutputMessage(id='msg_068a92f8b636a82100697072e88fe081938dc9d1400528e639', content=[ResponseOutputText(annotations=None, text='안녕 반가워!', type='input_text', logprobs=None)], role='user', status='completed', type='message'), ResponseCompactionItem(id='cmp_068a92f8b636a821016970761fc0608193a7f6032daabf257d', encrypted_content='gAAAAABpcHYkGwf1c29bnVFN6C0IBMAmm9jt8MwY3Gs9JRWc2Gg1g6x_6T4zLlSqrayV5CCEz6bwoWgPCYqgnwWqS9RGljqShtmD8W2iSqy2kCNVAKsN77SfVtiXRI77fwTtTosbDpYD55CqDodI2KO_8XKqyCqNkk5zbIreQOVPbAx-DsFvxHHE--9AUVkSmjoCjnY2exaL9paoCwy7OSGBf4BVeQT_QiLCoZcaXwWvfjNCXwNKeVx3NLNYdX0ZeCUjZ9KPbT4EyyPNR7RefFwcmJgDYKqBecRoCHi9TA3FR6DcLP954xfCunKBKNE5vEduJf68WvgozDH92dPeOUoYH_9IEPMc9CowCsg52hQjQipiR44Pw9VhhSQxMURfR7l4ACp413sw0ArLIlOQqbZ788go43UHtEPpvIlT9cCNNWSGkP4Df0rv5eY2CXr7xNaqPUp0ruEnBHZXw6PPaam_f5j9OrQAqb5_QB2M0q3EMZbYoapYnAB5KcUJ07K7aga_jSVe4dTZV